# Data

From the Angoulême in 1764 project:
https://histecon.fas.harvard.edu/visualizing/angouleme/overview.html

In [ ]:
df = read.csv("EDGESpublic6.11.20.csv")

In [ ]:
head(df)

## Step 1: Turn this into all unweighted, directed edges.

In [ ]:
edgelist = rbind(data.frame(source=df$Source, target=df$Target),
                 data.frame(source=df$Target[df$Type == 'Undirected'], target=df$Source[df$Type == 'Undirected']))

1. Number of edges?

2. Number of nodes?

Let's also grab the node data.

In [ ]:
nodes = read.csv("NODESpublic6.11.20.csv")

In [ ]:
head(nodes)

In [ ]:
nrow(nodes)

In [ ]:
max(nodes$ID)

## 1. Degree distribution

In [ ]:
degrees = c()
for (source in nodes$ID) {
    degrees=c(degrees, sum(edgelist$source == source))
}

In [ ]:
library(ggplot2)

In [ ]:
ggplot(data.frame(degrees), aes(degrees)) + geom_histogram()

In [ ]:
max(degrees)

In [ ]:
nodes[which.max(degrees),]

In [ ]:
df[df$Source == 3476,]

## 2. Shortest paths

In [ ]:
all_shortest_paths <- function(start, iters=Inf) {
    dists = rep(Inf, max(nodes$ID))
    dists[start] = 0
    queue = c(start)
    
    while (length(queue) > 0 && iters > 0) {
        ## Dequeue first item
        node = queue[1]
        queue = queue[-1]
        mydist = dists[node]
        
        ## Consider each neighbor
        for (ii in which(edgelist$source == node)) {
            if (dists[edgelist$target[ii]] > mydist + 1) {
                dists[edgelist$target[ii]] = mydist + 1
                queue = c(queue, edgelist$target[ii])
            }
        }
        
        iters = iters - 1
    }
    
    dists
}

In [ ]:
dists = all_shortest_paths(edgelist$source[1])

In [ ]:
dists[dists != Inf]

In [ ]:
meandists = c()
for (start in edgelist$source) {
    dists = all_shortest_paths(start)
    meandists = c(meandists, mean(dists[dists != 0 & dists != Inf]))
}

In [ ]:
meandists

In [ ]:
mean(meandists)

3. Clustering coefficient

In [ ]:
calc_clustering = function(node) {
    neighbors = edgelist$target[edgelist$source == node] # j in A_ij
    triangles = 0
    for (node2 in neighbors) {
        neighbors2 = edgelist$target[edgelist$source == node2] # k in A_jk
        for (node3 in neighbors2) {
            if (node %in% edgelist$target[edgelist$source == node3]) 
                triangles = triangles + 1
        }
    }
    triangles / (length(neighbors) * (length(neighbors) - 1))
}

In [ ]:
calc_clustering(edgelist$source[1])

In [ ]:
clustering = c()
for (node in edgelist$source) {
    cc = calc_clustering(node)
    clustering = c(clustering, cc)
}

In [ ]:
clustering

In [ ]:
mean(clustering[is.finite(clustering)])

## 3. Doing the work with igraph

In [ ]:
install.packages("igraph")

In [ ]:
library(igraph)

In [ ]:
net = graph_from_edgelist(as.matrix(edgelist), directed=T)

In [ ]:
net

In [ ]:
mean_distance(net)

In [ ]:
distances(net, mode="out")

In [ ]:
alldists = distances(net, mode="out")

In [ ]:
dists = alldists[edgelist$source[1],]

In [ ]:
dists[dists != Inf]

In [ ]:
mean(dists[dists != 0 & dists != Inf])

In [ ]:
meandists = apply(alldists, 1, function(dists) mean(dists[dists != 0 & dists != Inf]))

In [ ]:
meandists[edgelist$source[1]]

In [ ]:
mean(meandists, na.rm=T)

In [ ]:
transitivity(net)

In [ ]:
eigen_centrality(net)

In [ ]:
length(eigen_centrality(net)$vector)

In [ ]:
which.max(eigen_centrality(net)$vector)

In [ ]:
nodes[nodes$ID == 3476,]

In [ ]:
betweenness(net)

In [ ]:
which.max(betweenness(net))

In [ ]:
nodes[nodes$ID == 941,]

In [ ]:
df[df$Source == 941 | df$Target == 941,]

## 4. Infect someone

In [ ]:
infected = rep(F, max(nodes$ID))

In [ ]:
infected[941] = T

In [ ]:
epidemic = data.frame(time=0:1, infections=0:1)
while (epidemic$infections[nrow(epidemic)] != epidemic$infections[nrow(epidemic)-1]) {
    infected[edgelist$target[edgelist$source %in% which(infected)]] = T
    epidemic = rbind(epidemic, data.frame(time=epidemic$time[nrow(epidemic)]+1, infections=sum(infected)))
}

In [ ]:
ggplot(epidemic, aes(time, infections)) + geom_line()

In [ ]:
results = data.frame()
for (start in 1:max(nodes$ID)) {
    infected = rep(F, max(nodes$ID))
    infected[start] = T
    infections = 0
    while (sum(infected) > infections) {
        infections = sum(infected)
        infected[edgelist$target[edgelist$source %in% which(infected)]] = T
    }
    results = rbind(results, data.frame(start, infections))
}

In [ ]:
head(results)

In [ ]:
max(results$infections)

In [ ]:
which.max(results$infections)